I - import des données depuis le fichier data.py

In [1]:
from ml_in_finance_ensae.data import load_ff25_and_rf
ff25_excess, rf = load_ff25_and_rf()
ff25_excess.shape, rf.shape

((1194, 25), (1194,))

II - On procède à la régression des 25 portefeuilles sur les facteurs

L'objectif de cette partie est d'obtenir une première version simplifiée (linéaire) de la relation entre les portefeuilles et les facteurs. Quelques points intéressants à remarquer :


On a un modèle de  forme R_i,e,t = alpha_i + Beta_i.T * f_t + eps_i,t.
En posant M_t = 1 − b.⊤ * (f_t​ − μ_f​) on prouve (calculs à rédiger dans le papier) : E[M_t * ​R_i,t,e​] = α_i​.

Autrement dit, l'égalité à 0 recherchée (car M_t en tant que facteur d'actualisation doit vérifier l'égalité E[M_t * ​R_i,t,e​] = 0) s'applique aux α_i.

Dans les régressions ci-dessous, c'est une hypothèse qu'on peut rejeter pour 28% des régressions (on peut réfléchir à faire un test global qui sera rejeté a priori, mais ils n'en font pas vraiment dans le papier). Cela justifie l'idée qu'un modèle linéaire n'est pas suffisant pour capturer l'ensemble des excess returns offerts par le marché.

In [2]:
from ml_in_finance_ensae.data import load_ff25_and_rf, load_ff3_factors, DataPaths

ff25_excess, rf1 = load_ff25_and_rf(DataPaths())
ff3, rf2 = load_ff3_factors(DataPaths())

common = ff25_excess.index.intersection(ff3.index)
print(len(common), common.min(), common.max())
print(ff3.loc[common].head())

1194 1926-07-01 00:00:00 2025-12-01 00:00:00
            Mkt-RF     SMB     HML
Date                              
1926-07-01  0.0289 -0.0255 -0.0239
1926-08-01  0.0264 -0.0114  0.0381
1926-09-01  0.0038 -0.0136  0.0005
1926-10-01 -0.0327 -0.0014  0.0082
1926-11-01  0.0254 -0.0011 -0.0061


In [3]:
from ml_in_finance_ensae.data import load_ff25_and_rf, load_ff3_factors, DataPaths
from ml_in_finance_ensae.bench_ff3 import ff3_time_series_benchmark

ff25_excess, _ = load_ff25_and_rf(DataPaths())
ff3, _ = load_ff3_factors(DataPaths())

res = ff3_time_series_benchmark(ff25_excess, ff3)

res["summary"]
res["table"].sort_values("alpha_m").head(10)


,alpha_m,t_alpha,beta_Mkt-RF,beta_SMB,beta_HML
portfolio,,,,,
SMALL LoBM,-0.006736,-3.330116,1.268709,1.474734,0.350439
ME1 BM2,-0.003939,-3.314086,1.070705,1.529405,0.206005
ME5 BM4,-0.002412,-4.551584,1.031592,-0.179598,0.655390
ME2 BM1,-0.002138,-3.051768,1.085829,1.151121,-0.228246
ME4 BM5,-0.001737,-2.315476,1.185044,0.319525,0.946965
BIG HiBM,-0.001638,-1.632253,1.175891,-0.142561,0.996703
ME1 BM3,-0.001212,-1.405130,1.049362,1.237752,0.492476
ME3 BM1,-0.001190,-2.045613,1.123907,0.810971,-0.218582
ME3 BM5,-0.000621,-0.916865,1.112685,0.589237,0.864848


In [4]:
res["summary"]

{'n_obs': 1194,
 'start': Timestamp('1926-07-01 00:00:00'),
 'end': Timestamp('2025-12-01 00:00:00'),
 'rms_alpha_m': 0.0018472766622951932,
 'rms_alpha_ann': 0.02216731994754232,
 'share_|t_alpha|>2': 0.28}

In [5]:
table = res["table"]

# Ajouter |t_alpha|
table["abs_t_alpha"] = table["t_alpha"].abs()

# Top 5 par |t|
top5 = table.sort_values("abs_t_alpha", ascending=False).head(5)

top5[["alpha_m", "t_alpha"]]


,alpha_m,t_alpha
portfolio,,
ME5 BM4,-0.002412,-4.551584
SMALL LoBM,-0.006736,-3.330116
ME1 BM2,-0.003939,-3.314086
ME2 BM1,-0.002138,-3.051768
BIG LoBM,0.000980,2.981633


Ces résultats sont analysés plus haut.

On est quand même assez éloigné du papier encore ici donc on peut commencer à représenter le modèle d'une manière légèrement différente pour s'en rapprocher.

Hansen–Jagannathan distance : une mesure globale et le pont vers CPZ

La condition de base est toujours : E[M_t * ​R_t,e​] = 0.

Quand elle ne tient pas, on a un vecteur de "pricing errors" :
g = E[M_t * ​R_t,e​]

La question devient : à quel point ce g est grave ?
HJ propose de mesurer ça avec une norme "économiquement pertinente" :

d_HJ(M) =  sqrt(g.T * sigma_R-1 * g) où sigma_R = Var(R_t,e)

Cette quantité a une interprétation : c’est la plus petite dispersion (écart-type) d’un ajustement au SDF nécessaire pour corriger le pricing, dans un certain espace.

CPZ vont dans la même direction, mais au lieu de fixer la norme sigma_R-1, ils construisent une norme adversariale via des instruments g et un problème minimax.


III - Introduction d'une fonction g

On rappelle qu'on fait l'hypothèse que : 
M_t+1 = a - b.T * f_t+1.
Il faut noter que si M_t est un discount facteur admissible (E[M_t * ​R_i,t,e​] = 0) alors quel que soit le scalaire c, c * M_t l'est également.

Aussi, on peut simplement fixer a = 1 et choisir c qui permet de se ramener au M_t précédent (sans perte de généralité), ça simplifie la démarche.

In fine on va donc choisir b (i.e. M) en minimisant une norme de g_hat = 1/T * somme M_t * R_t_e qu'on peut encore écrire (et ce sera le problème à résoudre pour l'instant) :

g_hat(b) = 1/T * somme (t=1:T) de (1 - b.T * f_t) * R_t,e

In [6]:
from ml_in_finance_ensae.data import load_ff25_and_rf, load_ff3_factors, DataPaths
from ml_in_finance_ensae.sdf_linear import (
    estimate_lambda_from_time_series_betas,
    b_from_lambda,
    sdf_series,
    moment_vector,
    rms,
    top_violations,
)

ff25_excess, _ = load_ff25_and_rf(DataPaths())
ff3, _ = load_ff3_factors(DataPaths())

res = estimate_lambda_from_time_series_betas(ff25_excess, ff3)
b = b_from_lambda(ff3.loc[res["index"]], res["lambda"])

m = sdf_series(ff3.loc[res["index"]], b)
g = moment_vector(ff25_excess.loc[res["index"]], m)

print("b:\n", b)
print("RMS(g):", rms(g))
top_violations(g, k=10)


b:
 Mkt-RF    2.145190
SMB      -0.634784
HML       2.192165
Name: b, dtype: float64
RMS(g): 0.0016152952055141117


,g,abs_g
SMALL LoBM,-0.005341,0.005341
ME1 BM2,-0.002559,0.002559
ME5 BM4,-0.002495,0.002495
SMALL HiBM,0.002340,0.002340
BIG HiBM,-0.001757,0.001757
ME1 BM4,0.001674,0.001674
ME4 BM5,-0.001482,0.001482
ME3 BM2,0.001408,0.001408
ME2 BM2,0.001079,0.001079
BIG LoBM,0.001030,0.001030


Analyse des résultats précédents : 

Ici on choisit un b particulier en lien avec la régression FF3 précédente.

Précédemment, on a pour chaque portefeuille : E[R_i,e] = B_i.T * lambda. Dans un modèle bien spécifié on a b = sigma_f-1 * lambda où sigma_f = Var(ft).

Ici on a un RMS(g) = 0.00162, avec comme point de comparaison précédent RMS(alpha) = 0.00185 (même ordre de grandeur ce qui est le résultat souhaité).

Par ailleurs, les plus grosses violations interviennent pour les mêmes facteurs dans les deux approches. C'était attendu mais ça souligne que les approches sont deux manières différentes d'estimer le même modèle.

IV - Lien avec le problème minimax

Si on reste encore un peu éloigné du DL, on remarque que le problème minimax posé dans le papier s'apparente à : 

min [sur M_t dans un espace M] max [sur h, avec norme(h) <= 1 ] de 
E [h.T * M_t * R_t,e]

et le RMS(g) est égal à (pour un b donné) h.T * g(b)

A noter que ce problème et le problème de CPZ sont analogues, simplement ce problème évolue dans un espace plus restreint (notation un peu confuses mais h ici équivaut à g chez eux et b ici équivaut à w chez eux)


Transition avec le problème de CPZ

Depuis tout à l'heure on traite le problème : 

E[M_t * R_t,e] = 0.

Mais le véritable problème qu'on souhaite traiter est : 

E[M_t+1 * R_t+1,e | Z_t] = 0, où Z_t désigne l'information disponible au temps t.

On arrive alors à CPZ : 

E[Mt+1 * R_t+1,e * g(Zt)] = 0 pour tout g (c'est en ça que l'espace est plus large ici). Et c'est à présent que le DL prend tout son sens car pour estimer g proprement il n'y a pas vraiment d'autre solution...

V - Modèle de CPZ Simplifié

On repart du problème défini précédemment, mais avec h, pas avec g. Autrement dit, on veut traiter : 

min [sur b] max [sur h dans B2(1)] E[h.T * Mt(b) * R_t,e]

i.e. à b fixé : 

max [sur h dans B2(1)] h.T * g(b) = ||g(b)|| où ||.|| désigne la norme euclidienne.

Seulement, on reste ici sur un plan linéaire (pas de DL encore), ce qui rend le résultat plus lisible et simple.



In [7]:
from ml_in_finance_ensae.data import load_ff25_and_rf, load_ff3_factors, DataPaths
from ml_in_finance_ensae.sdf_linear import estimate_lambda_from_time_series_betas, b_from_lambda
from ml_in_finance_ensae.minimax_game import fit_minimax_b2, top_violations

ff25_excess, _ = load_ff25_and_rf(DataPaths())
ff3, _ = load_ff3_factors(DataPaths())

# b0 = b_FF3 (ce que tu avais déjà)
tmp = estimate_lambda_from_time_series_betas(ff25_excess, ff3)
b0 = b_from_lambda(ff3.loc[tmp["index"]], tmp["lambda"])

res = fit_minimax_b2(ff25_excess, ff3, b0=b0, n_steps=500, lr=0.5)

res["b"], res["history"].tail()
top_violations(res["g"], k=10)


,g,abs_g
SMALL LoBM,-0.005106,0.005106
SMALL HiBM,0.002553,0.002553
ME1 BM2,-0.002353,0.002353
ME5 BM4,-0.002341,0.002341
ME1 BM4,0.001868,0.001868
BIG HiBM,-0.001569,0.001569
ME3 BM2,0.001566,0.001566
ME4 BM5,-0.001277,0.001277
ME2 BM2,0.001257,0.001257
ME2 BM5,0.001188,0.001188


In [8]:
res["b"]

Mkt-RF    2.103756
SMB      -0.647714
HML       2.177732
Name: b, dtype: float64

In [9]:
res["history"].tail()

,step,norm_g,b_Mkt-RF,b_SMB,b_HML
495,495,0.008028,2.103758,-0.647734,2.177749
496,496,0.008028,2.103758,-0.647730,2.177746
497,497,0.008028,2.103757,-0.647726,2.177742
498,498,0.008028,2.103757,-0.647722,2.177739
499,499,0.008028,2.103757,-0.647718,2.177735


Ici, on obtient un b très proche du b_FF3 ce qui est positif. Cela indique que le problème que cherchent à résoudre CPZ est similaire au problème linéaire (lorsqu'on reste dans un monde linéaire).

On remarque aussi que les violations restent les mêmes (plus ou moins). Le problème ne vient donc pas de la manière dont on pose le problème, mais du qu'on réfléchit à partir d'une classe de SDF qui est trop pauvre.

Bilan (avant DL) : 

Nous avons reformulé l’évaluation d’un modèle factoriel en termes de SDF et de conditions de moments.
Un SDF linéaire en facteurs est choisi pour minimiser la pire violation de pricing possible, où l’adversaire sélectionne la combinaison d’actifs la plus mal pricée.
Cette procédure correspond à un jeu adversarial minimax entre un modeleur et un testeur.

Dans le cadre restreint d’un SDF linéaire FF3 et de moments inconditionnels, le jeu converge vers un équilibre proche du FF3 classique.
Les violations de pricing persistent pour certains portefeuilles (small/growth), indiquant une insuffisance structurelle de la classe de SDF, et non un problème d’optimisation.

Même lorsqu’il est optimisé contre un adversaire global, un SDF linéaire FF3 ne parvient pas à satisfaire les conditions de pricing pour l’ensemble des actifs.
Cela motive l’introduction de SDF plus flexibles et de moments conditionnels, comme dans Chen, Pelger et Zhu (2024).

In [1]:
from ml_in_finance_ensae.data import (
    DataPaths, load_ff3_factors, load_industry49, load_mom10, to_excess
)

ff3, rf = load_ff3_factors(DataPaths())
ind49 = load_industry49(DataPaths())
mom10 = load_mom10(DataPaths())

ind49_ex = to_excess(ind49, rf)
mom10_ex = to_excess(mom10, rf)

ind49.shape, mom10.shape, ind49_ex.shape, mom10_ex.shape


((1194, 49), (1188, 10), (1194, 49), (1188, 10))